In [1]:
from pathlib import Path
import librosa
import torch
from argparse import ArgumentParser
import matplotlib
import h5py
import tqdm
from IPython.display import Audio, display

import sys
sys.path += ['../src']

import utils
import wavenet_models
from utils import save_audio
from wavenet import WaveNet
from wavenet_generator import WavenetGenerator
from nv_wavenet_generator import NVWavenetGenerator
from nv_wavenet_generator import Impl

In [2]:
data = Path.cwd().parent.joinpath("musicnet", "parsed")

data = {label.stem : [wav for wav in data.joinpath(label).iterdir()] for label in data.iterdir()}

for key, value in data.items():
    print(key)
    print(value[0])

Beethoven_Accompanied_Violin
/home/wangjim19/orchestrator/music-translation/musicnet/parsed/Beethoven_Accompanied_Violin/2330.wav
Bach_Solo_Cello
/home/wangjim19/orchestrator/music-translation/musicnet/parsed/Bach_Solo_Cello/2217.wav
Bach_Solo_Piano
/home/wangjim19/orchestrator/music-translation/musicnet/parsed/Bach_Solo_Piano/2194.wav
Beethoven_Solo_Piano
/home/wangjim19/orchestrator/music-translation/musicnet/parsed/Beethoven_Solo_Piano/2322.wav
Beethoven_String_Quartet
/home/wangjim19/orchestrator/music-translation/musicnet/parsed/Beethoven_String_Quartet/2313.wav
Cambini_Wind_Quintet
/home/wangjim19/orchestrator/music-translation/musicnet/parsed/Cambini_Wind_Quintet/2075.wav


In [3]:
checkpoint = Path('../checkpoints/pretrained_musicnet/bestmodel_0.pth')

model_args = torch.load(checkpoint.parent / 'args.pth')[0]
encoder = wavenet_models.Encoder(model_args)
encoder.load_state_dict(torch.load(checkpoint)['encoder_state'])
encoder.eval()
encoder = encoder.cuda()

In [4]:
processed_data = {}
for label in data:
    processed_data[label] = []
    for path in data[label]:
        processed, rate = librosa.load(path, sr=16000)
        assert rate == 16000
        processed = utils.mu_law(processed)
        processed_data[label].append(torch.tensor(processed).unsqueeze(0).float().cuda())
        break
    break

In [5]:
for label in processed_data:
    processed_data[label] = torch.stack(processed_data[label]).contiguous()
    print(f'size: {processed_data[label].size()}')

size: torch.Size([1, 1, 8514665])


In [6]:
encoded_data = {}
with torch.no_grad():
    for label in processed_data:
        encoded_data[label] = []
        for sample in torch.split(processed_data[label], 1):
            encoded_data[label] += [encoder(sample)]
        encoded_data[label] = torch.cat(encoded_data[label], dim=0)
        break

RuntimeError: CUDA out of memory. Tried to allocate 4.06 GiB (GPU 0; 15.90 GiB total capacity; 12.22 GiB already allocated; 2.97 GiB free; 12.22 GiB reserved in total by PyTorch)